# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Import our modules and the Geoapify key
import hvplot.pandas
import pandas as pd
import requests
from api_keys import geoapify_key


In [2]:
# Load the WeatherPy CSV file into a Pandas DataFrame and display it
city_data_df = pd.read_csv("../output_data/BrownWeathercities.csv")
city_data_df.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fortuna,40.5982,-124.1573,15.52,94,0,0.00,US,1697424584
1,1,hwange,-18.3693,26.5019,25.01,61,96,0.59,ZW,1697424501
2,2,chonchi,-42.6128,-73.8092,6.18,98,70,1.20,CL,1697424584
3,3,gadzhiyevo,69.2551,33.3362,4.91,89,98,11.90,RU,1697424585
4,4,sitangkai,4.6628,119.3928,28.55,68,94,1.17,PH,1697424585


---

### Step 1: Create a map that displays a point for every city in the DataFrame, and set the point size to that city's humidity

In [3]:
%%capture --no-display

# Configure and display the map plot
humids=city_data_df.hvplot.points("Lon","Lat", geo=True, tiles="OSM", frame_width=1000,frame_height=1500,size="Humidity",color="City")
humids

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (City,Humidity)

### Step 2: Narrow down the DataFrame; what suits your fancy?

In [4]:
# Narrow down cities that fit your criteria
fittemp=(0,10)
fitwind=(0,4)
fitclouds=0

# Drop any rows with null values and display your destinations
perfect=city_data_df[(city_data_df["Max Temp"].between(fittemp[0],fittemp[1]))&(city_data_df["Wind Speed"].between(fitwind[0],fitwind[1]))&(city_data_df["Cloudiness"].fillna(0)<=fitclouds)].dropna()
perfect

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,stanley,54.8680,-1.6985,3.76,92,0,1.54,GB,1697424585
25,25,bethel,41.3712,-73.4140,9.31,82,0,0.00,US,1697424341
92,92,thompson,55.7435,-97.8558,4.09,93,0,1.03,CA,1697424597
171,171,bismarck,46.8083,-100.7837,9.34,75,0,2.57,US,1697424423
227,227,colonia,50.9333,6.9500,6.01,92,0,1.03,DE,1697424601
230,230,dryden,49.7833,-92.7503,4.16,87,0,2.57,CA,1697424619
304,304,karatau,43.1667,70.4667,6.59,52,0,1.48,KZ,1697424630
327,327,nurota,40.5614,65.6886,9.93,33,0,2.72,UZ,1697424634
340,340,mount sterling,38.0565,-83.9433,9.66,72,0,0.00,US,1697424636
375,375,papa,47.3300,17.4674,4.84,87,0,3.09,HU,1697424641


In [8]:
# Use the Pandas copy function to create a "hotels" DataFrame to store the city, country, coordinates, and humidity
hotels=perfect[["City","Country","Lat","Lon","Humidity"]].copy()

# Add an empty "Hotel Name" column to the DataFrame so you can store Geoapify's detected hotel, then display it
hotels["Hotel Name"]=""
hotels

,City,Country,Lat,Lon,Humidity,Hotel Name
9,stanley,GB,54.8680,-1.6985,92,
25,bethel,US,41.3712,-73.4140,82,
92,thompson,CA,55.7435,-97.8558,93,
171,bismarck,US,46.8083,-100.7837,75,
227,colonia,DE,50.9333,6.9500,92,
230,dryden,CA,49.7833,-92.7503,87,
304,karatau,KZ,43.1667,70.4667,52,
327,nurota,UZ,40.5614,65.6886,33,
340,mount sterling,US,38.0565,-83.9433,72,
375,papa,HU,47.3300,17.4674,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10 kilometres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
categories="accommodation.hotel"
params = {"categories":categories,"apiKey":geoapify_key}
# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotels.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lon"]  
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}" 
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make an API request using the params dictionary and convert the response to a JSON
    call = requests.get(base_url, params=params).json()    
    # Grab the first hotel from the results and store it in the hotels DataFrame
    try:
        hotels.loc[index, "Hotel Name"] = call["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotels.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotels.loc[index, 'City']} - nearest hotel: {hotels.loc[index, 'Hotel Name']}")

# Display 
hotels

Starting hotel search
stanley - nearest hotel: Hotel 52
bethel - nearest hotel: Hampton Inn Danbury
thompson - nearest hotel: Thompson Inn
bismarck - nearest hotel: Radisson Hotel Bismarck
colonia - nearest hotel: Wasserturm Hotel Cologne
dryden - nearest hotel: Holiday Inn Express
karatau - nearest hotel: Артабас
nurota - nearest hotel: Nurota
mount sterling - nearest hotel: Holiday Inn Express & Suites Mt Sterling North
papa - nearest hotel: Hotel Arany Griff
kolonia - nearest hotel: Wasserturm Hotel Cologne
brookings - nearest hotel: Quality Inn
south sioux city - nearest hotel: South Sioux City Marriott Riverfront
bilokurakyne - nearest hotel: Палуба
khovd - nearest hotel: Khovd hotel
pierre - nearest hotel: Quality Inn


,City,Country,Lat,Lon,Humidity,Hotel Name
9,stanley,GB,54.8680,-1.6985,92,Hotel 52
25,bethel,US,41.3712,-73.4140,82,Hampton Inn Danbury
92,thompson,CA,55.7435,-97.8558,93,Thompson Inn
171,bismarck,US,46.8083,-100.7837,75,Radisson Hotel Bismarck
227,colonia,DE,50.9333,6.9500,92,Wasserturm Hotel Cologne
230,dryden,CA,49.7833,-92.7503,87,Holiday Inn Express
304,karatau,KZ,43.1667,70.4667,52,Артабас
327,nurota,UZ,40.5614,65.6886,33,Nurota
340,mount sterling,US,38.0565,-83.9433,72,Holiday Inn Express & Suites Mt Sterling North
375,papa,HU,47.3300,17.4674,87,Hotel Arany Griff


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure and display
hotelmap=hotels.hvplot.points("Lon","Lat",geo=True,tiles="OSM",color="City",size="Humidity",hover_cols=["Hotel Name","Country"])
hotelmap

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (City,Humidity,Hotel Name,Country)